# Lecture 6: _k_-Nearest Neighbors

# Abalone Case Study: Classification with _k_-fold Cross Validation Approach
Source: _Machine Learning From Scratch_ (Machine Learning Mastery)

### Imports

In [ ]:
# python imports

from random import seed
from random import randrange
from csv import reader
from math import sqrt

### Function to load the CSV file and create a list of dataset


In [ ]:
# load a CSV file

def load_csv(filename):
	
	dataset = list()
 
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	 
	return dataset

### str_column_to_float() - converts string to numeric values 

In [ ]:
# convert string column to float

def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

### str_column_to_int()  -  output column is converted from string to integer values of 0 to 1

In [ ]:
# convert string column to integer

def str_column_to_int(dataset, column):

	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
 
	for i, value in enumerate(unique):
		lookup[value] = i
		
	for row in dataset:
		row[column] = lookup[row[column]]

	return lookup

### Find Min and Max values for each feature

In [ ]:
# Find the min and max values for each column

def dataset_minmax(dataset):

	minmax = list()
 
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	
	return minmax

### Rescale the dataset to the range between 0 to 1

In [ ]:
# Rescale dataset columns to the range 0-1

def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

### k-fold Cross Validation - estimate the performance of the model

In [ ]:
# Split a dataset into k folds

def cross_validation_split(dataset, n_folds):

	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
 
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	
	return dataset_split
  

### Estimate the Performance of _k_-models: accuracy

In [ ]:
# Calculate accuracy percentage

def accuracy_metric(actual, predicted):

	correct = 0

	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
			
	return correct / float(len(actual)) * 100.0

### Evaluate Classification Accuracy of Each Model Using Cross-Validation Split

In [ ]:
# Evaluate an algorithm using a cross validation split

def evaluate_algorithm(dataset, algorithm, n_folds, *args):

	folds = cross_validation_split(dataset, n_folds)
	scores = list()
 
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
	
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None

		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	
	return scores

### Calculate Distannce between two rows of data

In [ ]:
# Calculate the Euclidean distance between two vectors

def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

### Find _k_-most similar neighbors

In [ ]:
# Locate the most similar neighbors

def get_neighbors(train, test_row, num_neighbors):

	distances = list()
 
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
 
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	
	return neighbors


### Score the data with _k_-similar neighbors

In [ ]:
# Make a prediction with neighbors

def predict_classification(train, test_row, num_neighbors):

	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
 
	return prediction

# KNN on Abalone Dataset

### Run _k_-NN Algorithm: No Training 

In [ ]:
# kNN Algorithm

def k_nearest_neighbors(train, test, num_neighbors):

	predictions = list()
	for row in test:
		output = predict_classification(train, row, num_neighbors)
		predictions.append(output)
	
	return(predictions)

### Imports to load the csv file 

In [ ]:
# imports to upload the datafile
import io
from google.colab import files  

### Uploade the csv file to local directory

In [ ]:
# to upload local files
uploaded = files.upload()

Saving abalone.csv to abalone.csv


### Save the daset to working directory

In [ ]:
# load and prepare data
seed(1)
filename = 'abalone.csv'
dataset = load_csv(filename)

In [ ]:
dataset

### Prepare data

In [ ]:
# convert string class to float
for i in range(1, len(dataset[0])):
	str_column_to_float(dataset, i)
# convert first column to integers
str_column_to_int(dataset, 0)

{'F': 1, 'I': 2, 'M': 0}

### Score dataset and evaluate accuracy across k-folds

In [ ]:
# evaluate algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [24.790419161676645, 21.79640718562874, 23.592814371257482, 21.676646706586826, 23.353293413173652]
Mean Accuracy: 23.042%
